In [41]:
import numpy as np

In [42]:
class MDP_DP():
    
    def __init__(self):
        
        # actions are list or array. States, V, R and policy are dictionaries with keys being the states
        self.actions = actions 
        self.states = states
        self.V = {s:0 for s in self.states} #dictionary
        self.R = R 
        self.policy = policy
        # next_state is a funcction that takes in initial state and action as tuple to return next state
        self.next_state = next_state 
        # gamma is the discount rate
        self.gamma = gamma
        # the states in which there shouldn't be any policy improvement
        self.terminal_states = terminal_states
        
    
    def policy_evaluation(self):
        
        # function that loops through all states and applies bellman equation to calculate their value
        # delta is a small number used to test for convergence of the value of states
        
        delta = 0.00000001
        d = 1000
        
        while d > delta:
            
            V_new = self.V.copy()
            for i in self.V:
                v = 0
                for action in self.actions:
                    j = self.next_state(i, action)
                    v += self.policy[i][action]*(self.R[i] + self.gamma*self.V[j])                
                V_new[i] = v
            
            d = self.get_delta(self.V,V_new)
            self.V = V_new
            
                   
    def get_delta(self,v,vnew):
        
        # function that returns the absolute value of the mean difference between previous and current values of a state
        d = 0
        for i in v:
            d += abs(v[i]-vnew[i])/len(self.V)
        return d
    
    
    def policy_improvement(self):
        
        # function that changes the policy in accordance to the value of the next states
        stable = False
        while stable == False:
            stable = True
            for i in self.policy:
                new_policy = np.zeros(len(self.actions))
                
                pi_s = []
                for action in self.actions: 
                    j = self.next_state(i, action)
                    pi_s.append(self.V[j])
                    
                new_policy = self.get_new_policy(pi_s,i)

                if not np.all(new_policy == self.policy[i]):
                    stable = False
                    self.policy[i] = new_policy
                    
            if not stable:
                self.policy_evaluation()
                
    def get_new_policy(self, val, state):
        
        #function that takes in values of states to return the ones with highest value
        if state in self.terminal_states:
            new_policy = np.zeros(4)
        else:
            maximum = max(val)
            indeces = []
            for i in range(len(val)):
                if val[i] == maximum:
                    indeces.append(i)
            new_policy = np.zeros(len(val))
            new_policy[indeces] = 1
            new_policy = np.array([x/new_policy.sum() for x in new_policy])
        
        return new_policy

In [43]:
class Grid(MDP_DP):
    def __init__(self, shape):
        
        self.shape = shape #tuple
        self.actions = [x for x in range(4)]
        self.R = np.ones((shape))*-1
        self.R[0,0], self.R[shape[0]-1,shape[1]-1]  = 0, 0
        #self.V = np.zeros(shape)
        
    
    def get_v(self):
        v={}
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                v[(i,j)] = 0
#         v[0,0] = 0
#         v[self.shape[0]-1, self.shape[1]-1] = 0
        
        self.V = v
    
    def get_policy(self):
        policy = {}
        for i in self.V:
            policy[i] = np.ones(len(self.actions))*(1/len(self.actions))
        policy[(0,0)] = np.zeros(len(self.actions))
        policy[(self.shape[0]-1,self.shape[0]-1)] = np.zeros(len(self.actions))
        
        self.policy = policy
        
    def get_reward(self):
        r= {}
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                r[(i,j)] = -1
        r[0,0] = 0
        r[self.shape[0]-1, self.shape[1]-1] = 0
        
        self.R = r
        
        
        
        
    
    def next_state(self, x, a):
        actions = [[1,0],[-1,0],[0,1],[0,-1]]
        move = actions[a]
        x_new, y_new = x[0] + move[0], x[1] + move[1]
        if x_new == self.shape[0]:
            x_new = x[0]
        if x_new < 0:
            x_new = x[0]
        if y_new == self.shape[1]:
            y_new = x[1]
        if y_new < 0:
            y_new = x[1]
    
        return (x_new, y_new) 

    

In [44]:
my_grid = Grid((4,4))
my_grid.get_v()
my_grid.get_policy()
my_grid.get_reward()
my_grid.gamma = 1
my_grid.terminal_states = [(0,0),(3,3)]
my_grid.policy_evaluation()

In [45]:
my_grid.policy_improvement()

In [46]:
my_grid.V

{(0, 0): 0.0,
 (0, 1): -1.0,
 (0, 2): -2.0,
 (0, 3): -3.0,
 (1, 0): -1.0,
 (1, 1): -2.0,
 (1, 2): -3.0,
 (1, 3): -2.0,
 (2, 0): -2.0,
 (2, 1): -3.0,
 (2, 2): -2.0,
 (2, 3): -1.0,
 (3, 0): -3.0,
 (3, 1): -2.0,
 (3, 2): -1.0,
 (3, 3): 0.0}

In [47]:
my_grid.policy

{(0, 0): array([0., 0., 0., 0.]),
 (0, 1): array([0., 0., 0., 1.]),
 (0, 2): array([0., 0., 0., 1.]),
 (0, 3): array([0.5, 0. , 0. , 0.5]),
 (1, 0): array([0., 1., 0., 0.]),
 (1, 1): array([0. , 0.5, 0. , 0.5]),
 (1, 2): array([0.25, 0.25, 0.25, 0.25]),
 (1, 3): array([1., 0., 0., 0.]),
 (2, 0): array([0., 1., 0., 0.]),
 (2, 1): array([0.25, 0.25, 0.25, 0.25]),
 (2, 2): array([0.5, 0. , 0.5, 0. ]),
 (2, 3): array([1., 0., 0., 0.]),
 (3, 0): array([0. , 0.5, 0.5, 0. ]),
 (3, 1): array([0., 0., 1., 0.]),
 (3, 2): array([0., 0., 1., 0.]),
 (3, 3): array([0., 0., 0., 0.])}